## 2. Preparação para Tokenização (Pré-processamento)

* **Input**: CSV contendo todos os enunciados, alternativas e gabaritos

* **Output**: CSV com as colunas pré-processadas -> *numero_questao*, *enunciado*, *alternativas*,*nu_param_B*, *gabarito*, *ano*, *enunciado_limpo* e *alternativas_limpo*

In [3]:
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords

In [4]:
# Download stopwords (palavras de parada)
nltk.download('stopwords')
stop_words = set(stopwords.words('portuguese'))

def clean_text(text):
    """Processando texto de acordo com o Protocolo Primi (2021), citado no artigo"""
    if not isinstance(text, str):  # Garantir que texto é string
        return ""
    
    words = re.findall(r'\b[a-zA-Zà-úÀ-ÚüÜ]+\b', text.lower())  # 1. Dividir item em palavras
    words = [word for word in words if word not in stop_words]  # 2. Remover stopwords
    return " ".join(sorted(set(words)))  # 3. Remover palavras duplicadas

def clean_alternatives(alt_text):
    """Processa as alternativas mantendo as letras (A:, B:, etc.) e limpando o conteúdo."""
    if not isinstance(alt_text, str):
        return ""
    
    alternatives = re.split(r'(?=[A-E]: )', alt_text)  # Divide mantendo os identificadores
    cleaned_alts = []
    
    for alt in alternatives:
        if ": " in alt:
            key, value = alt.split(": ", 1)  # Separa a letra da alternativa do conteúdo
            cleaned_value = clean_text(value)  # Aplica limpeza somente no conteúdo
            cleaned_alts.append(f"{key}: {cleaned_value}")  # Mantém o formato original
    
    return "; ".join(cleaned_alts)  # Junta novamente as alternativas

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/gustavo/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
df = pd.read_csv("../data/final/enem_data.csv")

# Aplicando pré-processamento no enunciado e alternativas
df["enunciado_limpo"] = df["enunciado"].apply(clean_text)
df["alternativas_limpo"] = df["alternativas"].apply(clean_alternatives)

# Save cleaned CSV
df.to_csv("../data/cleaned_data.csv", index=False)
print("Processing complete. File saved as 'cleaned_data.csv'.")


Processing complete. File saved as 'cleaned_data.csv'.
